In [1]:
from bs4 import BeautifulSoup as bs
import requests as r
import pandas as pd

In [2]:
# https://www.megamillions.com/Winning-Numbers/Previous-Drawings.aspx
url = 'https://www.megamillions.com/jackpot-history'

In [3]:
def read_html(url, element):
    data = r.get(url)
    html_parse = bs(data.content, 'html.parser')
    return html_parse.find_all(element)


def clean_html(string):
    html_dict = {'\xa0':' ',
                'won by':'-',
                'won in':'-',
                '–':'-'
                }
    
    for key,val in html_dict.items():
        string = string.replace(key,val)
    return(string)


def html_to_df(url):
    df = pd.DataFrame(columns = ['date','amount','winner'])
    index = 0
    winners = read_html(url, 'li')
    for winner in winners:
        win_am = winner.text
        if 'million' in win_am or 'billion' in win_am:
            try:
                df.loc[index] = clean_html(win_am).split(' - ')
            except ValueError:
                df.loc[index] = win_am.replace('$','- ').split(' - ')
            index += 1
    return df


In [4]:
jackpot_winners = html_to_df(url)
jackpot_winners.to_csv('jackpot_winners.csv', index=False)